## Sign Language Detection Overview

Our program focuses on taking a clear image of a hand which displays a letter from the Irish Sign Language alphabet. We then take that hand and inform the user on which letter they are displaying. In order to keep our program away from machine learning and focus only on image processing we are detecting the most distinct letters which in this case are A,W,L,C. 
The final version of the program uses a webcam to crop the area where the user has their hand and informs which sign they were 
displaying. 


To get these results we used the following techniques: resize, to change the images to the same size as our tempaltes, we used segmentation in order to get the ROI using inRange and morphology,canny for edge detection, countouring to get the contours using our canny, sorted contours based on area which allowed us to make a bounding rectangle around the biggest contour. This allowed us to use the rectangle to get the width and height of each sign, finally we would use these dimensions to compare with user images in order to determine which sign they are displaying.

## References 

Template Matching 

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_template_matching/py_template_matching.html

Skin Detection

https://www.pyimagesearch.com/2014/08/18/skin-detection-step-step-example-using-python-opencv/

Canny ( Edge detection )

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html

Shape Matching 

https://docs.opencv.org/master/d5/d45/tutorial_py_contours_more_functions.html

Contours

https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contours_begin/py_contours_begin.html


## What was used 
Overall our main program uses canny and contouring creating a rectangle using a convex hull to determine what shape is being detected however we referenced template matching and shape matching because we also used it through out our development of the program but because of certain issues they did not make it to the final program, we talk more about this in our presentation. 

## Instructions
The program offers two option, 1) Input from an Image 2) input from webcam. For the first option when chosing your image. You need to select an image from your computer that you want to check for sign language. Make sure when uploading an image of the hand,the image as an appropriate background and appropriate distance from the camera. You can use the template and as example of what the image should look like. After you upload the image the program will display what sign you were displaying (program works for the detection of 4 signs A, C, W and L ). 

The Second option opens up your camera, if you do not have a camera the program will give out. After the camera opens please insert you hand with the sign language sign that you wish to detect (out of A, C, L and W ). Once you get the hand into the rectangle press S. After pressing S the program captures the image inside the square and then presents you with the result.

## Detecting Sign from Image
Code used to determine the sing from an image provided by the user 

In [1]:
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#Author: Maciej Skrzypczynski, Paul Lanigan, Cormac Smith 
#Date : 27/11/2020
#Description : This program is used to determine ISL alphabet letters from images, The letters we are detecting for are A, C, W and L. We get this done with the skin detection to detect the
# hand from the images, then we use canny to get all the edges around the hand. After we get the edges we create contours using the edges we have detected. After we have the contours.
#We sort them from biggest to smallest and we use the biggest contours and bound a rectangle around it to determine its width and height. Finally we compare the high and width from our
#template to the users image to determine which sign is being displayed. Due to the vast similarity among the ISL alphabet the program works for A C W and L because they were the most 
# destinct letters in the ISL alphabet.

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as image
import easygui

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
cap = cv2.VideoCapture(0)

def make_1080p():
    cap.set(3, 1920)
    cap.set(4, 1080)
    
make_1080p()


        
#Read in templates of sign language and convert to HSV
aTemplate = cv2.imread('aTemplate.png')
cTemplate = cv2.imread('cTemplate.png')
lTemplate = cv2.imread('lTemplate.png')
wTemplate = cv2.imread('wTemplate.png')


UserInput = ""

# Text GUI 

while(True):
    print("Welcome to the Program Please Select from the following Options")
    print("1) Open up the Camera ")
    print("2) Enter image from computer")
    print("WARNING! When opening the camera Make sure to Press S once your hand is inside the rectangle")
    print("After you press S the program will take your hand and give you appropriate output\n ")
    UserInput = input("Enter input ....")
    if UserInput == "1" or "2":
        break
    else:
        print("Invalid input try again ....")

#when userinput is one open up camera 
if UserInput == "1":
    #Open up the camera 
    while(True):
        # Capture frame-by-frame
        ret, frame = cap.read()
        frame = cv2.rectangle(frame, (50,50), (353,470), (255,0,0), 1)

        # Display the resulting frame
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('s'): 

            check, frame = cap.read()
            cv2.imshow("Capturing", frame)
            userImage = frame[50:470,50:353]
            cv2.imwrite(filename='userImage.jpg', img=userImage)
            cap.release()
            userImage = cv2.imread('userImage.jpg')
            cv2.destroyAllWindows()

            break
#Select file from windows when userinput is 2 
if UserInput == "2":
    f = easygui.fileopenbox()
    userImage = cv2.imread(f)



#Resize the user image to be the same dimention as the templates 
dim = (503, 670)
userImage = cv2.resize(userImage, dim, interpolation = cv2.INTER_AREA)

#Convert Images to HSV 
hsvA = cv2.cvtColor(aTemplate,cv2.COLOR_BGR2HSV)
hsvC = cv2.cvtColor(cTemplate,cv2.COLOR_BGR2HSV)
hsvL = cv2.cvtColor(lTemplate,cv2.COLOR_BGR2HSV)
hsvW = cv2.cvtColor(wTemplate,cv2.COLOR_BGR2HSV)
hsvI = cv2.cvtColor(userImage,cv2.COLOR_BGR2HSV)

#skin detection ranges 
min_HSV = np.array([0, 48, 80], dtype = "uint8")
max_HSV = np.array([20, 255, 255], dtype = "uint8")

#create the mask that just contains the skin ( hand )
skinMaskA = cv2.inRange(hsvA, min_HSV, max_HSV)
skinMaskC = cv2.inRange(hsvC, min_HSV, max_HSV)
skinMaskL = cv2.inRange(hsvL, min_HSV, max_HSV)
skinMaskW = cv2.inRange(hsvW, min_HSV, max_HSV)
skinMaskUser = cv2.inRange(hsvI, min_HSV, max_HSV)

#using morphology adjusting the mask to be more accurate 
shape = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))

#for templates 
closingA = cv2.morphologyEx(skinMaskA,cv2.MORPH_CLOSE,shape)
openingA = cv2.morphologyEx(closingA,cv2.MORPH_OPEN,shape)

closingC = cv2.morphologyEx(skinMaskC,cv2.MORPH_CLOSE,shape)
openingC = cv2.morphologyEx(closingC,cv2.MORPH_OPEN,shape)

closingL = cv2.morphologyEx(skinMaskL,cv2.MORPH_CLOSE,shape)
openingL = cv2.morphologyEx(closingL,cv2.MORPH_OPEN,shape)

closingW = cv2.morphologyEx(skinMaskW,cv2.MORPH_CLOSE,shape)
openingW = cv2.morphologyEx(closingW,cv2.MORPH_OPEN,shape)

#for user input
closingUser = cv2.morphologyEx(skinMaskUser,cv2.MORPH_CLOSE,shape)
openingUser = cv2.morphologyEx(closingUser,cv2.MORPH_OPEN,shape)

# Get ROIs of the hands for all the templates 
ROIA = cv2.bitwise_and(aTemplate,aTemplate,mask=openingA)
ROIC = cv2.bitwise_and(cTemplate,cTemplate,mask=openingC)
ROIL = cv2.bitwise_and(lTemplate,lTemplate,mask=openingL)
ROIW = cv2.bitwise_and(wTemplate,wTemplate,mask=openingW)
ROIUser = cv2.bitwise_and(userImage,userImage,mask=openingUser)

#Initilizing my Hull List
hull_listA = []
hull_listC = []
hull_listL = []
hull_listW = []
hull_listUser = []

# Get all of the edges using the ROI 

#for templates 
aE = cv2.Canny(ROIA, 30, 150)
cE = cv2.Canny(ROIC, 30, 150)
lE = cv2.Canny(ROIL, 30, 150)
wE = cv2.Canny(ROIW, 30, 150)

#for user input
userE = cv2.Canny(ROIUser, 30, 150)



#Use canny to find contours

#for templates 
contoursA,_ =cv2.findContours(aE, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
hull_listA = sorted(contoursA, key=cv2.contourArea, reverse=True)
contoursC,_ =cv2.findContours(cE, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
hull_listC = sorted(contoursC, key=cv2.contourArea, reverse=True)
contoursL,_ =cv2.findContours(lE, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
hull_listL = sorted(contoursL, key=cv2.contourArea, reverse=True)
contoursW,_ =cv2.findContours(wE, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
hull_listW = sorted(contoursW, key=cv2.contourArea, reverse=True)

#for user input
contoursUser,_ =cv2.findContours(userE, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
hull_listUser = sorted(contoursUser, key=cv2.contourArea, reverse=True)

#Drawing the contours 

#for templates 
imgA = cv2.drawContours(aTemplate, hull_listA[0], -1, (0,0,255), -1)
imgC = cv2.drawContours(cTemplate, hull_listC[0], -1, (0,255,0), -1)
imgL = cv2.drawContours(lTemplate, hull_listL[0], -1, (0,0,255), -1)
imgW = cv2.drawContours(wTemplate, hull_listW[0], -1, (0,255,0), -1)



# Creating bounding rectangle around the countours and calculating the total widths and heights 

#for templates 
tw = 0
th = 0
x,y,w,h=cv2.boundingRect(hull_listA[0])

tw = tw + w
th = th + h
x,y,w,h=cv2.boundingRect(hull_listL[0])

tw = tw + w
th = th + h
x,y,w,h=cv2.boundingRect(hull_listC[0])

tw = tw + w
th = th + h
x,y,w,h=cv2.boundingRect(hull_listW[0])

tw = tw + w
th = th + h

#for user input
xUser,yUser,wUser,hUser=cv2.boundingRect(hull_listUser[0])

averageW = (tw / 4) + 25
averageH = (th / 4) + 25

#check for sign from users input by comparing the dimentions 
if hUser < averageH:
    if wUser < averageW:
        print("The Image provided most likely displays The A sign")
    else:
        print("The Image provided most likely displays The C sign")

if hUser > averageH:
    if wUser > averageW:
        print("The Image provided most likely displays The L sign")
    else:
        print("The Image provided most likely displays The W sign")



#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Welcome to the Program Please Select from the following Options
1) Open up the Camera 
2) Enter image from computer
WARNING! When opening the camera Make sure to Press S once your hand is inside the rectangle
After you press S the program will take your hand and give you appropriate output 
The Image provided most likely displays The A sign


## Conclusion

Overall, we planned the program to detect Irish Sign Language alphabet from an image and then later from the camera. We managed to read in the images and get images from the camera. The issue we encountered is that the skin detection seems to be really difficult to master so the image / webcam need to be against an appropriate background in order to secure an accurate result. Because a lot of letters in the ISL are very similar we had to chose images that looked the most distinct, which in our case were the signs for A, L ,C and W. Another issue that we had is that we could not find any other way of giving the result without stepping into machine learning, even after researching online most solutions used machine learning, the only way we found it to work is as follows. We got the contours and then created bounded rectangles around the contours because the images are distinct from each other some turn out the be wider / longer then others which means that by those dimensions we are able to differentiate between the images. However this solution only works when the user keeps a similar distance when taking an image / using the webcam as per the templates and also the background has to be appropriate.